<a href="https://colab.research.google.com/github/Lambda-School-Labs/bridges-to-prosperity-ds-b/blob/feature%2Fmodel_prototype_nb/B2P_Labs28_Model_prototyping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np

In [15]:
from google.colab import files
uploaded = files.upload()

KeyboardInterrupt: ignored

In [16]:
df = pd.read_csv('b2p_cleaned.csv', index_col=0)
df.head()

,Bridge Name,Bridge Opportunity: Level 1 Government,Bridge Opportunity: Level 2 Government,Bridge Opportunity: Project Code,Bridge Opportunity: Needs Assessment,Bridge Opportunity: GPS (Latitude),Bridge Opportunity: GPS (Longitude),Bridge Opportunity: Bridge Type,Bridge Opportunity: Stage,Bridge Opportunity: Individuals Directly Served,Bridge Opportunity: Comments,Bridge Opportunity: Span (m),Current crossing methond,Nearest all-weather crossing point,Bridge Classification,Flag for Rejection,Form: Form Name,Form: Created By,Rejection Reason,Height differential between banks,Senior Engineering Review Conducted,Market access blocked by river,Education access blocked by river,Health access blocked by river,Other access blocked by river,Days per year river is flooded,Flood duration during rainy season,River crossing injuries in last 3 years,River crossing deaths in last 3 years,Primary Occupations,Primary crops grown,4WD Accessibility,cell service quality
0,Bukinga,Southern Province,Nyaruguru,1009317,Rwanda Needs Assessment 2018,-2.760833,29.488056,Suspended Bridge,Cancelled,NaN,Steep slopes to deal with for anchors,NaN,NaN,NaN,NaN,Yes,Project Assessment - 2018.10.10,aimablengirabakunzi taroworks,NaN,NaN,Yes,0.0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Kagarama,Western Province,Nyamasheke,1007325,NaN,-2.322534,29.141944,Suspension Bridge,Complete,4000.0,NaN,48.0,NaN,2km downstream,NaN,NaN,Project Assessment - 2017.7.12,Stephanie May,NaN,0.7,Yes,0.0,0,0,0,120.0,NaN,15.0,2.0,NaN,banana,Seasonally,NaN
2,Karambi,Southern Province,Huye,1007374,NaN,-2.513333,29.612778,Suspension Bridge,Complete,12500.0,Likely piloting steel decking on this bridge. ...,50.0,NaN,Is located at 5km upstream. People pay 1000rwf...,NaN,NaN,Project Assessment - 2018.7.27,etiennemutebutsi taroworks,NaN,1.0,Yes,0.0,1,1,1,60.0,NaN,15.0,3.0,NaN,"Corns, banana and beans",Never,NaN
3,Rugeti,Southern Province,Nyamagabe,1007466,Rwanda Needs Assessment 2018,-2.476407,29.420800,Suspension Bridge,Complete,4363.0,"3000-6000 people directly served, Elevation:21...",68.0,NaN,None,NaN,No,Project Assessment - 2018.9.1,aimablengirabakunzi taroworks,NaN,1.1,Yes,0.0,1,1,1,60.0,NaN,2.0,0.0,NaN,"Tea, cow pea, wheat, beans, sweet potatoes",Seasonally,NaN
4,Nyakabuye - Nkomane,Southern Province,Nyamagabe,1007495,Rwanda Needs Assessment 2018,-2.233022,29.575130,Suspension Bridge,Complete,2790.0,"Not Duplicate 500-1000 people directly served,...",78.0,NaN,"Vehicular bridge, 5 kilometers",NaN,No,Project Assessment - 2018.9.4,dominiquehabimana taroworks,NaN,1.1,Yes,0.0,1,1,1,35.0,NaN,4.0,4.0,NaN,"Cassava, maize, beans, sorghum, sweet potatoes",Never,NaN


In [17]:
bridges_df = df.copy()

In [19]:
# Any sites with a "Yes" in the column "Senior Engineering Review Conducted"
# have undergone a full technical review, and of those, the 
# "Bridge Opportunity: Stage" column can be considered to be correct.
positive = (
    (bridges_df['Senior Engineering Review Conducted']=='Yes') & 
    (bridges_df['Bridge Opportunity: Stage'].isin(['Complete', 'Prospecting', 'Confirmed']))
)

negative = (
    (bridges_df['Senior Engineering Review Conducted']=='Yes') & 
    (bridges_df['Bridge Opportunity: Stage'].isin(['Rejected', 'Cancelled']))
)

# Any sites without a "Yes" in column Senior Engineering Review Conducted" 
# have not undergone a full technical review ...
# So these sites are unknown and unlabeled
unknown = bridges_df['Senior Engineering Review Conducted'].isna()

# Create a new column named "Good Site." This is the target to predict.
# Assign a 1 for the positive class and 0 for the negative class.
bridges_df.loc[positive, 'Good Site'] = 1
bridges_df.loc[negative, 'Good Site'] = 0

# Assign -1 for unknown/unlabled observations. 
# Scikit-learn's documentation for "Semi-Supervised Learning" says, 
# "It is important to assign an identifier to unlabeled points ...
# The identifier that this implementation uses is the integer value -1."
# We'll explain this soon!
bridges_df.loc[unknown, 'Good Site'] = -1

In [20]:
bridges_df['Good Site'].value_counts()

-1.0    1383
 1.0      65
 0.0      24
Name: Good Site, dtype: int64

In [21]:
bridges_df = bridges_df.drop(columns=['Senior Engineering Review Conducted',
                                      'Bridge Opportunity: Stage',
                                      'Rejection Reason', 'Bridge Opportunity: Comments',
                                      'Bridge Opportunity: Project Code'])
bridges_df.head()

,Bridge Name,Bridge Opportunity: Level 1 Government,Bridge Opportunity: Level 2 Government,Bridge Opportunity: Needs Assessment,Bridge Opportunity: GPS (Latitude),Bridge Opportunity: GPS (Longitude),Bridge Opportunity: Bridge Type,Bridge Opportunity: Individuals Directly Served,Bridge Opportunity: Span (m),Current crossing methond,Nearest all-weather crossing point,Bridge Classification,Flag for Rejection,Form: Form Name,Form: Created By,Height differential between banks,Market access blocked by river,Education access blocked by river,Health access blocked by river,Other access blocked by river,Days per year river is flooded,Flood duration during rainy season,River crossing injuries in last 3 years,River crossing deaths in last 3 years,Primary Occupations,Primary crops grown,4WD Accessibility,cell service quality,Good Site
0,Bukinga,Southern Province,Nyaruguru,Rwanda Needs Assessment 2018,-2.760833,29.488056,Suspended Bridge,NaN,NaN,NaN,NaN,NaN,Yes,Project Assessment - 2018.10.10,aimablengirabakunzi taroworks,NaN,0.0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,Kagarama,Western Province,Nyamasheke,NaN,-2.322534,29.141944,Suspension Bridge,4000.0,48.0,NaN,2km downstream,NaN,NaN,Project Assessment - 2017.7.12,Stephanie May,0.7,0.0,0,0,0,120.0,NaN,15.0,2.0,NaN,banana,Seasonally,NaN,1.0
2,Karambi,Southern Province,Huye,NaN,-2.513333,29.612778,Suspension Bridge,12500.0,50.0,NaN,Is located at 5km upstream. People pay 1000rwf...,NaN,NaN,Project Assessment - 2018.7.27,etiennemutebutsi taroworks,1.0,0.0,1,1,1,60.0,NaN,15.0,3.0,NaN,"Corns, banana and beans",Never,NaN,1.0
3,Rugeti,Southern Province,Nyamagabe,Rwanda Needs Assessment 2018,-2.476407,29.420800,Suspension Bridge,4363.0,68.0,NaN,None,NaN,No,Project Assessment - 2018.9.1,aimablengirabakunzi taroworks,1.1,0.0,1,1,1,60.0,NaN,2.0,0.0,NaN,"Tea, cow pea, wheat, beans, sweet potatoes",Seasonally,NaN,1.0
4,Nyakabuye - Nkomane,Southern Province,Nyamagabe,Rwanda Needs Assessment 2018,-2.233022,29.575130,Suspension Bridge,2790.0,78.0,NaN,"Vehicular bridge, 5 kilometers",NaN,No,Project Assessment - 2018.9.4,dominiquehabimana taroworks,1.1,0.0,1,1,1,35.0,NaN,4.0,4.0,NaN,"Cassava, maize, beans, sorghum, sweet potatoes",Never,NaN,1.0


In [28]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(bridges_df, train_size=0.90, test_size=0.10, stratify=bridges_df['Good Site'], random_state=42)

train, val = train_test_split(train, train_size=0.80, test_size=0.20, stratify=train['Good Site'], random_state=42)

train.shape, val.shape, test.shape

((1059, 29), (265, 29), (148, 29))

In [29]:
target = 'Good Site'

train_features = train.drop(columns=[target])

numeric_features = train_features.select_dtypes(include='number').columns.tolist()

cardinality = train_features.select_dtypes(exclude='number').nunique()
categorical_features = cardinality[cardinality <= 50].index.tolist()

features = numeric_features + categorical_features
print(features)

['Bridge Opportunity: GPS (Latitude)', 'Bridge Opportunity: GPS (Longitude)', 'Bridge Opportunity: Individuals Directly Served', 'Bridge Opportunity: Span (m)', 'Current crossing methond', 'Bridge Classification', 'Height differential between banks', 'Market access blocked by river', 'Education access blocked by river', 'Health access blocked by river', 'Other access blocked by river', 'Days per year river is flooded', 'Flood duration during rainy season', 'River crossing injuries in last 3 years', 'River crossing deaths in last 3 years', 'Primary Occupations', 'cell service quality', 'Bridge Opportunity: Level 1 Government', 'Bridge Opportunity: Level 2 Government', 'Bridge Opportunity: Needs Assessment', 'Bridge Opportunity: Bridge Type', 'Flag for Rejection', 'Form: Created By', '4WD Accessibility']


In [30]:
X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]
y_test = test[target]

In [33]:
!pip install category_encoders==2.*

     |████████████████████████████████| 81kB 2.2MB/s 


In [35]:
import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.semi_supervised import LabelSpreading
from sklearn.pipeline import make_pipeline

LS_pipe = make_pipeline(ce.ordinal.OrdinalEncoder(), SimpleImputer(),
                       LabelSpreading(kernel='knn', n_jobs=-1))

LS_pipe.fit(X_train, y_train)
print(f'Train accuracy: {LS_pipe.score(X_train, y_train)}')
print(f'Validation accuracy: {LS_pipe.score(X_val, y_val)}')

/usr/local/lib/python3.6/dist-packages/sklearn/semi_supervised/_label_propagation.py:293: RuntimeWarning: invalid value encountered in true_divide
  self.label_distributions_ /= normalizer


Train accuracy: 0.05476864966949953
Validation accuracy: 0.033962264150943396


In [36]:
LS2_pipe = make_pipeline(ce.ordinal.OrdinalEncoder(), SimpleImputer(),
                       LabelSpreading(kernel='rbf', n_jobs=-1))

LS2_pipe.fit(X_train, y_train)
print(f'Train accuracy: {LS2_pipe.score(X_train, y_train)}')
print(f'Validation accuracy: {LS2_pipe.score(X_val, y_val)}')

Train accuracy: 0.0169971671388102
Validation accuracy: 0.01509433962264151


/usr/local/lib/python3.6/dist-packages/sklearn/semi_supervised/_label_propagation.py:293: RuntimeWarning: invalid value encountered in true_divide
  self.label_distributions_ /= normalizer


In [37]:
from sklearn.semi_supervised import LabelPropagation

LP_pipe = make_pipeline(ce.ordinal.OrdinalEncoder(), SimpleImputer(),
                       LabelPropagation(kernel='knn', n_jobs=-1))

LP_pipe.fit(X_train, y_train)
print(f'Train accuracy: {LP_pipe.score(X_train, y_train)}')
print(f'Validation accuracy: {LP_pipe.score(X_val, y_val)}')

/usr/local/lib/python3.6/dist-packages/sklearn/semi_supervised/_label_propagation.py:277: RuntimeWarning: invalid value encountered in true_divide
  self.label_distributions_ /= normalizer
/usr/local/lib/python3.6/dist-packages/sklearn/semi_supervised/_label_propagation.py:288: ConvergenceWarning: max_iter=1000 was reached without convergence.
  category=ConvergenceWarning


Train accuracy: 0.01794145420207743
Validation accuracy: 0.01509433962264151


In [38]:
LP2_pipe = make_pipeline(ce.ordinal.OrdinalEncoder(), SimpleImputer(),
                       LabelPropagation(kernel='rbf', n_jobs=-1))

LP2_pipe.fit(X_train, y_train)
print(f'Train accuracy: {LP2_pipe.score(X_train, y_train)}')
print(f'Validation accuracy: {LP2_pipe.score(X_val, y_val)}')

/usr/local/lib/python3.6/dist-packages/sklearn/semi_supervised/_label_propagation.py:277: RuntimeWarning: invalid value encountered in true_divide
  self.label_distributions_ /= normalizer


Train accuracy: 0.0169971671388102
Validation accuracy: 0.01509433962264151


/usr/local/lib/python3.6/dist-packages/sklearn/semi_supervised/_label_propagation.py:288: ConvergenceWarning: max_iter=1000 was reached without convergence.
  category=ConvergenceWarning
